# 1. Estructura Medallon (Bronze → Silver → Gold)

%md
**Bronze (Raw Layer)**

- Datos crudos, tal como vienen de la fuente.
- Pueden estar en CSV, JSON, Parquet, logs, etc.
- Objetivo: almacenamiento inmutable y trazabilidad.

%md
**Silver (Cleansed Layer)**

- Datos limpios, estandarizados y con esquema definido.
- Se aplican: validaciones, casting de tipos, joins básicos.
- Objetivo: datos listos para análisis internos.

%md
**Gold (Business Layer)**

- Datos enriquecidos y agregados.
- Orientados a dashboards, reporting y ML.

%md
## 1) `fleet_dim_aircraft.csv` (tabla de dimensión)
Catálogo maestro de aeronaves.

- **aircraft_id** *(STRING)*: Identificador interno único del avión (clave para joins).
- **tail_number** *(STRING)*: Matrícula de la aeronave (p. ej., EC-XXX).
- **model** *(STRING)*: Modelo (p. ej., A320-214, A321neo).
- **manufacturer** *(STRING)*: Fabricante (p. ej., Airbus).
- **engine_model** *(STRING)*: Modelo de motor (p. ej., CFM56-5B4).
- **first_service_dt** *(DATE)*: Fecha de entrada en servicio.
- **status** *(STRING)*: Estado operativo (Active | Maintenance | Retired).
- **cycles_total** *(INT)*: Ciclos acumulados (despegue/aterrizaje).
- **hours_total** *(DOUBLE)*: Horas de vuelo acumuladas.

**Relación:** Se une con las tablas de hechos por `aircraft_id`.

---

## 2) `ops_fact_engine_sensor.csv` (tabla de hechos – telemetría simplificada)
Muestras de telemetría (baja resolución para demo) y **12 columnas**.

- **reading_id** *(STRING)*: Identificador único de la lectura (UUID).
- **aircraft_id** *(STRING)*: FK a `fleet_dim_aircraft.aircraft_id`.
- **event_ts** *(TIMESTAMP, UTC en ISO8601)*: Marca temporal del evento.
- **event_date** *(DATE)*: Fecha derivada de `event_ts` (útil para particionado).
- **engine_pos** *(TINYINT)*: Posición del motor (1=izq., 2=dcha.).
- **phase_of_flight** *(STRING)*: Fase del vuelo (Taxi/Takeoff/Climb/Cruise/Descent/Landing).
- **altitude_ft** *(INT)*: Altitud estimada en pies.
- **ias_kts** *(INT)*: Indicated Airspeed (nudos).
- **egt_c** *(DOUBLE)*: Exhaust Gas Temperature (°C).
- **n1_pct** *(DOUBLE)*: % de RPM del fan (N1).
- **fuel_flow_kg_h** *(DOUBLE)*: Caudal de combustible (kg/h).
- **vib_ips** *(DOUBLE)*: Vibración en pulgadas/segundo.

**Uso típico:** generación de *features* de ventana por `aircraft_id` + `event_ts` (p. ej., medias móviles de `egt_c`, `vib_ips`, `n1_pct`).

---

## 3) `mro_fact_maintenance_event_50k.csv` (tabla de hechos – 50k filas)
Histórico de eventos de mantenimiento, inspecciones y correctivos.

- **event_id** *(STRING)*: Identificador del evento (UUID).
- **aircraft_id** *(STRING)*: FK a `fleet_dim_aircraft.aircraft_id`.
- **event_ts** *(TIMESTAMP, UTC en ISO8601)*: Fecha y hora del evento.
- **event_date** *(DATE)*: Fecha derivada (útil para particionado).
- **event_type** *(STRING)*: Tipo de evento (Inspection | A-Check | Unscheduled | Corrective | Replacement).
- **ata_chapter** *(STRING)*: Capítulo ATA (sistema afectado, p. ej., “72 Engine”).
- **defect_code** *(STRING)*: Código interno de defecto.
- **severity** *(STRING)*: Severidad percibida (Low | Medium | High | Critical).
- **grounded** *(BOOLEAN)*: Si el avión quedó en tierra (AOG) por el evento.
- **workorder_id** *(STRING)*: Orden de trabajo en el sistema MRO/ERP.
- **duration_min** *(INT)*: Duración de la intervención (minutos).
- **description** *(STRING)*: Resumen libre del trabajo/defecto.
- **next_due_cycles** *(INT, nullable)*: Ciclos restantes hasta el siguiente check (si aplica).
- **next_due_hours** *(DOUBLE, nullable)*: Horas restantes hasta el siguiente check (si aplica).

**Uso típico:** sirve como **etiqueta/target** para modelos supervisados (p. ej., probabilidad de evento “Unscheduled/Corrective” en ≤ X horas).

---

## Recomendaciones de modelado
- **Particionado por fecha** en tablas de hechos → `event_date`.
- **Z-Ordering** (Delta Lake) sobre (`aircraft_id`, `event_ts`) para acelerar consultas por avión/ventana temporal.
- **Calidad de datos**: Validar rangos físicos (e.g., `egt_c`, `n1_pct`) y coherencia de fases de vuelo.
- **Features**: estadísticas móviles (5/15/60 min), deltas vs. baseline del avión, *lags* y *rolling std* sobre `vib_ips` y `egt_c`.


In [0]:
%sql
--drop catalog if exists sesion1_2 cascade;
drop schema if exists sesion1_2.layer20_bronze cascade;
drop schema if exists sesion1_2.layer30_silver cascade;
drop schema if exists sesion1_2.layer30_silver_quarantine cascade;

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS sesion1_2;

-- 01 · Landing / Raw
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer10_landing;
CREATE VOLUME IF NOT EXISTS sesion1_2.layer10_landing.volume;

-- 02 · Bronze
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer20_bronze;

-- 03 · Silver
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer30_silver;
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer30_silver_quarantine;

-- 04 · Gold
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer40_gold;

-- 99 · Operaciones / Configuración
CREATE SCHEMA IF NOT EXISTS sesion1_2.layer99_ops;
CREATE VOLUME IF NOT EXISTS sesion1_2.layer99_ops.configs;


#2 Objetivo de la sesión

Con el YAML exportado al repositorio, desplegaremos el Workflow en Databricks, adaptando su configuración para que funcione correctamente.

In [0]:
%sql
select * 
from sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine

aircraft_id,tail_number,model,manufacturer,engine_model,first_service_dt,status,cycles_total,hours_total
AC1000,TN-1000,A320,Airbus,CFM56,2018-01-15,ACTIVE,12000,23500.5
AC1000,TN-1000B,A320,Airbus,CFM56,2018-06-01,ACTIVE,12100,23600.0
null,TN-NULL,A319,Airbus,CFM56,2017-03-10,INACTIVE,8000,15000.0
,TN-BLANK,B737,Boeing,LEAP-1B,2019-09-09,ACTIVE,9000,17000.0
AC2001,TN-2001,A321,Airbus,LEAP-1A,2020-02-02,ACTIVE,-5,10000.0
AC2002,TN-2002,B787,Boeing,GEnx,2021-11-20,ACTIVE,5000,250000.0
AC2004,TN-2004,A320neo,Airbus,LEAP-1A,2020-08-08,ACTIVE,110000,12000.0
AC2005,TN-2005,B737 MAX,Boeing,LEAP-1B,2021-01-01,ACTIVE,2000,-1.0


# 3. Creando Metricas

In [0]:
%sql

-- Vista KPI de % en cuarentena antes y después del fix
CREATE OR REPLACE VIEW sesion1_2.layer40_gold.v_quarantine_rate_dim_aircraft AS
WITH
-- Total de registros válidos
valid_dim AS (
  SELECT COUNT(*) AS cnt_valid
  FROM sesion1_2.layer30_silver.dim_aircraft
),

-- Total de registros en cuarentena y su estado de fix
q_dim AS (
  SELECT
    COUNT(*) AS cnt_q_total,
    SUM(CASE WHEN COALESCE(fixed, FALSE) THEN 1 ELSE 0 END) AS cnt_q_fixed,
    SUM(CASE WHEN COALESCE(fixed, FALSE) THEN 0 ELSE 1 END) AS cnt_q_remaining
  FROM sesion1_2.layer30_silver_quarantine.dim_aircraft_quarantine
)

-- Cálculo final del KPI
SELECT
  'dim_aircraft' AS table_name,
  v.cnt_valid,
  q.cnt_q_total,
  q.cnt_q_fixed,
  q.cnt_q_remaining,
  (v.cnt_valid + q.cnt_q_total) AS denom_total,
  ROUND(
    CASE WHEN (v.cnt_valid + q.cnt_q_total) = 0
         THEN 0.0
         ELSE (q.cnt_q_total * 100.0) / (v.cnt_valid + q.cnt_q_total)
    END, 2
  ) AS pct_quarantine_before,
  ROUND(
    CASE WHEN (v.cnt_valid + q.cnt_q_total) = 0
         THEN 0.0
         ELSE (q.cnt_q_remaining * 100.0) / (v.cnt_valid + q.cnt_q_total)
    END, 2
  ) AS pct_quarantine_after
FROM valid_dim v
CROSS JOIN q_dim q;
